In [361]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
import pandas as pd
from config2 import api_key
from pprint import pprint
import openweathermapy.core as owm
import seaborn as sns

In [419]:
import numpy as np

np_lat = np.arange(-90,90,10)
np_lng = np.arange(-180,180,18)
# coordinates = (np_lat,np_lng) #Better keeping lat/lng in separate columns

import random

rnd_lat = random.choice(np_lat)
rnd_lng = random.choice(np_lng)
# rnd_coordinates = random.choice(coordinates) #Better keeping lat/lng in separate columns

from citipy import citipy

weather_df = pd.DataFrame()
weather_df['rnd_lat']=""
weather_df['rnd_lng']=""

for x in np_lat:
    for y in np_lng:
        x_values = list(np.arange(-90,90,0.01))
        y_values = list(np.arange(-180,180,0.01))
        lats = random.sample(x_values,17999) #why 17999 instead of 500? Because of duplicates..
        lons = random.sample(y_values,17999) #..stemming from the geocoords from citipy
                                            #and that's the maximum number of samples I can input
                                            #given the latitude parameters (180/0.01 = 18000)
                                            #Also I believe a larger sampling size to be better to
                                            #increase the random variability of values.
        
weather_df = weather_df.append(pd.DataFrame.from_dict({"rnd_lat":lats,
                                       "rnd_lng":lons}))

weather_df = weather_df.reset_index(drop=True)
# print(weather_df.shape)

# print(weather_df.head())


weather_df['city']=""
weather_df['country_code']=""

for index,row in weather_df.iterrows():
        weather_df_city = citipy.nearest_city(row["rnd_lat"], row["rnd_lng"])
        weather_df.set_value(index,"city",weather_df_city.city_name)
        weather_df.set_value(index,"country_code",weather_df_city.country_code)
        
weather_df=weather_df.drop(['rnd_lat', 'rnd_lng'],axis=1) #geo coord don't correspond to cities

weather_df=weather_df.drop_duplicates() #hmmm, some coords might yield similar cities....
print("Sampling size (from n = 17,999) after dumping duplicates: " + str(len(weather_df)))

weather_df = weather_df.sample(500) #resample so that n=500
weather_df = weather_df.reset_index(drop=True) #Remove index pre-resampling

print(weather_df.head())

print("Confirm n = 500: " + str(weather_df.shape))

C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:42: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\White Base\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:43: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Sampling size (from n = 1,799) after dumping duplicates: 3324
             city country_code
0  krasnaya zarya           ru
1          maputo           mz
2           sapao           ph
3        old road           ag
4          mirina           gr
Confirm n = 500: (500, 2)


In [414]:
# API Configuration

target_url = 'http://api.openweathermap.org/data/2.5/weather?'

units = "metric" # https://openweathermap.org/current ; API Documentation
settings = {"units": "metric", "appid": api_key,}
city = list(weather_df['city'])
country_code = list(weather_df['country_code'])


# Build partial query URL
query_url = f"{target_url}appid={api_key}&units={units}&q={city}&q={country_code}"
query_url=requests.get(query_url, params = settings).json()

pprint(query_url)

In [325]:
# attributes = ('city', 'country_code', 'rnd_lat', 'rnd_lng', 'citipy_lat', 'citipy_lng','Temp (°C)',
#             'Humidity (%)','Cloudiness (%)','Wind Speed (mph)')
# weather_df = pd.DataFrame(columns = attributes)

In [ ]:
 if city_name not in cities_df.city_name.values:
            print('Status code: %s. DF length is now: %d' % (str(weather_json["cod"]), len(cities_df)+1))